In [2]:

!pip install datasets
!pip install sentencepiece
!pip install datasets
!pip install transformers
!pip install torch
from datasets import load_dataset
import pandas as pd
import csv
from datasets import load_dataset
import torch as th
from transformers import T5Tokenizer, T5Model,T5ForConditionalGeneration,T5Config
from transformers import get_scheduler
from tqdm.auto import tqdm
from torch.utils.data import DataLoader
from transformers import AdamW
import torch

train_dataset = load_dataset("jfleg", split='validation[:]')

eval_dataset = load_dataset("jfleg", split='test[:]')
def generate_csv(csv_path, dataset):
    with open(csv_path, 'w', newline='') as csvfile:
        writter = csv.writer(csvfile)
        writter.writerow(["input", "target"])
        for case in dataset:
     	    # Adding the task's prefix to input 
            input_text = "grammar: " + case["sentence"]
            for correction in case["corrections"]:
                # a few of the cases contain blank strings. 
                if input_text and correction:
                    writter.writerow([input_text, correction])
                    


generate_csv("train.csv", train_dataset)
generate_csv("eval.csv", eval_dataset)
train_dataset=pd.read_csv('./train.csv')
eval_dataset=pd.read_csv('./eval.csv')

<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#5642C5;
           font-size:30px;
           font-family:Verdana;
           letter-spacing:0.5px">
               Text Preprocessing

</div>

In [3]:

replacements = [
  (" .", "."), 
  (" ,", ","),
  (" '", "'"),
  (" ?", "?"),
  (" !", "!"),
  (" :", "!"),
  (" ;", "!"),
  (" n't", "n't"),
  (" v", "n't"),
  ("2 0 0 6", "2006"),
  ("5 5", "55"),
  ("4 0 0", "400"),
  ("1 7-5 0", "1750"),
  ("2 0 %", "20%"),
  ("5 0", "50"),
  ("1 2", "12"),
  ("1 0", "10"),
  ('" ballast water', '"ballast water')
]

def remove_excess_spaces(text):
  for rep in replacements:
    text = text.replace(rep[0], rep[1])

  return text
sentence=pd.Series(train_dataset['input']).map(lambda x : remove_excess_spaces(x))
correction=pd.Series(train_dataset['target']).map(lambda xx : remove_excess_spaces(xx) )
task_prefix = "grammar: "

train=pd.DataFrame(columns=['sent','corr'])

for i in range(len(sentence)):
        train=train.append({'sent': sentence[i],'corr':correction[i]}, ignore_index=True)
            
    
qns = list(train["sent"])
print(len(qns)) 


class GSMDataset(th.utils.data.Dataset):
    def __init__(self, tokenizer,dataset, loss_on_prefix=True):
        self.examples = dataset
        self.qns = list(self.examples["sent"])
        self.ans = list(self.examples["corr"])
        self.qns = tokenizer(self.qns,padding='max_length', max_length=418, truncation=True, return_tensors="pt")
        self.ans = tokenizer(self.ans,padding='max_length',max_length=405, truncation=True)
        
        self.loss_on_prefix = loss_on_prefix

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        qn_tokens = self.qns["input_ids"][idx]
        qn_att=self.qns["attention_mask"][idx]
        ans_tokens = self.ans["input_ids"][idx]
        ans_tokens = [-100 if x==0 else x for x in ans_tokens] 
        qn_tokens = th.tensor(qn_tokens)
        qn_att = th.tensor(qn_att)
        ans_tokens=th.tensor(ans_tokens)
        
        return dict(input_ids= qn_tokens, attention_mask=qn_att,labels=ans_tokens)
    

<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#5642C5;
           font-size:30px;
           font-family:Verdana;
           letter-spacing:0.5px">
               Tokenization and parameter initilization

</div>

In [4]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")
config = T5Config.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small",config=config,)
train_set= GSMDataset(tokenizer,train)
model.train()
train_loader = DataLoader(train_set, batch_size=8, shuffle=True)
optim = AdamW(model.parameters(), lr=1e-5)

num_epochs = 10
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optim,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

)


<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#5642C5;
           font-size:30px;
           font-family:Verdana;
           letter-spacing:0.5px">
               Model Training 

</div>

In [ ]:
pbar = tqdm(range(num_training_steps))
for epoch in range(num_epochs):
        for batch in train_loader:
            optim.zero_grad()
            b_input_ids,b_attention_mask,b_labels= batch.items()
            outputs = model(input_ids=batch["input_ids"],attention_mask=batch["attention_mask"], labels=torch.tensor(batch["labels"]))
            loss = outputs[0]
            loss.backward()
            optim.step()
            lr_scheduler.step()
            pbar.update(1)
            pbar.set_description(f"train_loss: {loss.item():.5f}")
            model.save_pretrained("model_ckpts/"

<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#5642C5;
           font-size:30px;
           font-family:Verdana;
           letter-spacing:0.5px">
               Prediction phase
</div>

In [ ]:
model_path = "/content/model_ckpts"
model = T5ForConditionalGeneration.from_pretrained(model_path,config=config,)
inputs = tokenizer('grammar:i loving writing articles',padding='max_length', max_length=418, truncation=True, return_tensors="pt").input_ids
ans_tokens = [-100 if x==0 else x for x in list(inputs[0])]
o=model.generate(torch.tensor(inputs))
tokenizer.decode(o[0],skip_special_tokens=True, clean_up_tokenization_spaces=True)